# Imports & Pipeline Initialization/

In [1]:
import mlflow
from transformers import pipeline
import ipywidgets as widgets
from IPython.display import display, clear_output

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
print("Summarization pipeline loaded successfully.")

Device set to use cpu


Summarization pipeline loaded successfully.


In [2]:
def run_summarization(text, run_name="SummarizationRun", max_length=200, min_length=100):
    """
    Summarizes the provided text using the initialized summarization pipeline.
    Wraps the inference in an MLflow run and logs parameters and metrics.
    
    Parameters:
      text (str): The text to summarize.
      run_name (str): The name for the MLflow run.
      max_length (int): Maximum length of the summary.
      min_length (int): Minimum length of the summary.
    
    Returns:
      str: The generated summary.
    """
    try:
        # Start an MLflow run with the provided run name.
        mlflow.start_run(run_name=run_name)
        mlflow.log_param("model", "facebook/bart-large-cnn")
        mlflow.log_param("max_length", max_length)
        mlflow.log_param("min_length", min_length)
        
        # Summarization
        result = summarizer(text, max_length=max_length, min_length=min_length, do_sample=False)
        summary_text = result[0]['summary_text']
        
        mlflow.log_metric("summary_length", len(summary_text))
        
        #End run
        mlflow.end_run()
        return summary_text
    except Exception as e:
        mlflow.end_run()
        print("An error occured during summarization:", str(e))
        return None

# Static Summarization Example

In [5]:
text_to_summarize = """
Federal Funds Target Rate Range
The Fed influences interest rates by monitoring and changing the target range for the federal funds rate (the overnight rate at which banks lend reserves to each other).
It usually sets a 25 basis point range, such as 5.25%-5.50%, which helps maintain a desirable effective federal funds rate (EFFR).
The EFFR is a volume-weighted median of loans between these depository institutions.
This rate influences all other rates, including those for bank loans and credit card balances. As a result, it also influences spending and saving, which affects the amount of money circulating throughout the economy.
Interest on Reserve Balances
In the past the Fed influenced the money supply by modifying reserve requirements. This refers to the amount of funds banks are required to hold against deposits in bank accounts.
The Fed no longer requires banks to hold reserves. Its primary tool is now interest on reserve balances (IORB). By paying interest on any reserves that banks keep, it establishes a certain level of support for rates. This keeps the federal funds rate from dropping too far below it.
IORB influences banks to keep money in reserve or deplete their reserves based on demand for loans and the level of rates—adding or subtracting to the supply of circulating money.
The Discount Rate
Banks can borrow money from the Fed using a lending program it calls the discount window. The interest rate set for these loans helps set the top number (the ceiling) for the federal funds rate target range. These loans are short-term, up to 90 days.
By lowering (or raising) the discount rate that banks pay on short-term loans from the Federal Reserve Bank, the Fed effectively increases (or decreases) the liquidity of the banking system.
Overnight Reverse Repurchase Agreements
The Federal Reserve conducts overnight reverse repurchase (ON RRP) agreements, in which it sells a security to an institution, then buys it back the next day for more money. The interest rate used for ON RRPs helps the Fed set the lower rate (the floor) of its fed funds target range.
These reverse repos subtract money from reserves, in essence taking money out of circulation.
Open Market Operations
In open market operations, the Fed purchases and sells securities issued by the U.S. government (such as Treasuries), which can affect the amount of money in circulation.
Open market operations once played a major role in the implementation of the Fed's monetary policy. Currently, they're conducted only to help the central bank maintain the "ample level of reserves" it believes is needed to continue to administer the aforementioned rates to influence the effective federal funds rate.
"""

static_summary = run_summarization(text_to_summarize, run_name="StaticTextSummarization")
if static_summary:
    print("Static summary generated")
    print(static_summary)

Static summary generated
The Fed influences interest rates by monitoring and changing the target range for the federal funds rate (EFFR) It usually sets a 25 basis point range, such as 5.25%-5.50%, which helps maintain a desirable effectiveFederal Funds Target Rate Range. The discount rate that banks pay on short-term loans from the Fed increases (or decreases) the liquidity of the banking system. The interest rate used for ON RRPs helps the Fed set the lower rate (the floor) of its fed funds target range.


# Interactive Summarization with User Input

In [7]:
user_input_text = widgets.Textarea(
    value='',
    placeholder="Paste or type text here...",
    description='Input Text:',
    disabled=False,
    layout=widgets.Layout(width='100%', height='200px')
)

summarize_button = widgets.Button(
    description="Summarize Text",
    button_style='success',
)

output_area = widgets.Output()

In [8]:
def on_summarize_clicked(b):
    with output_area:
        clear_output()
        text = user_input_text.value
        if not text.strip():
            print("Please enter text to summarize.")
        else:
            summary = run_summarization(text, run_name="UserTextSummarization")
            if summary:
                print("Summary:")
                print(summary)
            else:
                print("Failed to generate summary.")
                
summarize_button.on_click(on_summarize_clicked)

display(user_input_text, summarize_button, output_area)

Textarea(value='', description='Input Text:', layout=Layout(height='200px', width='100%'), placeholder='Paste …

Button(button_style='success', description='Summarize Text', style=ButtonStyle())

Output()